<a href="https://colab.research.google.com/github/CPallaresU/TALLERES-CEIA/blob/main/SEMANA_4ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
import pandas as pd

In [73]:
class model_class(object):

  def __init__(self):
      self.model = None

  def fit(self, X, Y):
      return NotImplemented

  def predict(self, X):
      return NotImplemented

class linear_Reg (model_class):

  def fit(self,x,y):
     w = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y) #utilizamos esta formula a partir de despejar el parametro W del error cuadratico medio 
     self.weight=w                                 #igualado a cero y derivado, obteniendo el optimo W
     return self.weight

  def predict(self,x,parameters):
    
    y=np.dot(x,parameters)
    return y

In [74]:
class metric(object):

  def __init__(self,y_actual,y_pred,TP,FP,TN,FN):
    self.y_actual=y_actual
    self.y_pred=y_pred
    self.TP = TP
    self.FP = FP
    self.TN = TN
    self.FN = FN

  def vals_(self):

    for i in range(len(self.y_pred)): 
        if self.y_actual[i]==self.y_pred[i]==1:
           self.TP += 1
        if self.y_pred[i]==1 and self.y_actual[i]!=self.y_pred[i]:
           self.FP += 1
        if self.y_actual[i]==self.y_pred[i]==0:
           self.TN += 1
        if self.y_pred[i]==0 and self.y_actual[i]!=self.y_pred[i]:
           self.FN += 1
    return self.TP,self.FP,self.TN,self.FN


class mse(metric):
  
  def mse_(self):
    return (sum(np.square(self.y_actual-self.y_pred))/len(self.y_pred))

class recall(metric):

  def recall_(self):
    return self.TP / (self.TP + self.FN)

class precision(metric):

  def precision_(self):
    return self.TP / (self.TP + self.FP)

In [75]:
def remove_NaN(X,axis_):
  if axis_==1: #Filas
    t=~np.isnan(X).any(axis=axis_)
    #X=X.T
    X=X[t]
    #X=X.T
  else: #Columnas
    t=~np.isnan(X).any(axis=axis_)
    X=X[t]
  
  return X

def split_data(data,random_v):

  data=data.sample(frac=.5, replace=True, random_state=random_v)
  x,y=np.shape(data)
  train=data.iloc[:int(x*.7)]
  validation=data.iloc[int(x*.7):int(x*.9)]
  test=data.iloc[int(x*.9):]

  return np.array(train),np.array(validation),np.array(test)

##Leemos los datos y los normalizamos ya que la variable charges es la variable objetivo e inicilamente tenia valores sobre 10.000, entonces trabajar con atributos entre 1 y 100 para obtener valores tan altos genera mucho ruido y los parametros deberan tomar valores muy garndes, por lo que normalizamos todo el dataset

In [76]:
!git clone https://github.com/CPallaresU/TALLERES-CEIA.git
q=pd.read_csv("/content/TALLERES-CEIA/insurance.csv")

q['sex'].replace(['female', 'male'],
                        [0, 1], inplace=True)

q['smoker'].replace(['yes', 'no'],
                        [1, 0], inplace=True)
q['region'].replace(['southwest', 'northwest', 'northeast', 'southeast'],
                        [1, 2, 3, 4], inplace=True)

#q['charges']=np.abs((q['charges']-q['charges'].mean())/q['charges'].std())
q=np.abs((q-q.mean())/q.std())

fatal: destination path 'TALLERES-CEIA' already exists and is not an empty directory.


In [77]:
w=remove_NaN(np.array(q),1)
df = pd.DataFrame(w, columns = q.columns)

##Generamos ahora diferentes regresiones con semillas diferentes para el split

In [81]:
seeds= [25,226,42,5222]

for i in seeds:
  train,validation,test=split_data(df,i)

  model=linear_Reg()
  parameters=model.fit(train[:,:np.shape(train)[1]-1],
            train[:,-1]) # y = x[:,np.shape(x)[1]-1] selecciona la ultima columna, el -1 es para no salir de las n-1 dimensiones

  y_pred=model.predict(test[:,:np.shape(test)[1]-1],parameters)
  error = mse(test[:,-1],y_pred,0,0,0,0)
  print("Error asociado a la predicción: {}".format(mse.mse_(error)))
  print(parameters)

Error asociado a la predicción: 0.26909421106602727
[ 0.05909184  0.12873523 -0.03152183 -0.02276854  0.74588858  0.00938263]
Error asociado a la predicción: 0.2631841891059832
[ 1.15858793e-04  2.06102537e-01 -4.96361936e-02 -6.48287344e-02
  8.01698039e-01  1.83217146e-02]
Error asociado a la predicción: 0.29967393868218684
[-0.0082769   0.01829268 -0.02666655  0.00935314  0.77133828  0.18692245]
Error asociado a la predicción: 0.20285488913635374
[ 0.0485297   0.16731061  0.09077543 -0.05211508  0.69518935 -0.06146418]
